### 🚀 Challenge #A1: Eigene Knowledge Base

Als Basis für meinen Knowledge Graph habe ich die NieR (Computerspiel-) Serie genommen und die wesentlichste Entitäten mit ihren Beziehungen gezeichnet:

![](./nier_knowledge_graph.png)

### 🚀 Challenge #A2: OpenAI GPT <3 Google Knowledge Graph

Manche (wenn nicht alle) APIs fordert einen API-Schlüssel, um damit interagieren zu können. Diese werden in der Regel in einer separaten Datei gespeichert, die zur Laufzeit wie folgt gelesen werden:

In [79]:
import json

# Set up your API keys
GOOGLE_KG_API_KEY = 'your_google_kg_api_key'
OPENAI_API_KEY = 'your_openai_api_key'

# Keeping secrets secret (。・ω・。)
with open('./secrets.json', 'r') as file:
    keys = json.load(file)
    GOOGLE_KG_API_KEY = keys['GOOGLE']
    OPENAI_API_KEY = keys['OPENAI']

Um mit dem Thema fortzufahren, definiere ich eine Anfrage zu meiner ausgewählten NieR Serie:

In [80]:
query = "NieR:Automata"

#### Google Knowledge Graph

Um Anfragen an Google KG API stellen zu können, kann die Request-URL verwendet werden:
```
https://kgsearch.googleapis.com/v1/entities:search?query=YOUR_QUERY&key=API_KEY&limit=1&indent=True
```

**Wichtig:** man muss die Knowledge Graph Dienst explizit für den Projekt aktivieren, dem der API-Schlüssel zugewiesen ist. Wenn nicht der Fall, alle Anfragen geben den Hinweis mit dem Link zum Portal zurück.

Den URL kann man mit Python `requests` Bibliothek ein bisschen schöner bauen und für die Anfrage verwenden:

In [81]:
import requests

# The function to query Google Knowledge Graph
def query_knowledge_graph(query):
    url = "https://kgsearch.googleapis.com/v1/entities:search"
    params = {
        'query': query,
        'limit': 1,
        'indent': True,
        'key': GOOGLE_KG_API_KEY
    }
    response = requests.get(url, params=params)
    data = response.json()
    if 'itemListElement' in data and len(data['itemListElement']) > 0:
        entity = data['itemListElement'][0]['result']
        return entity.get('name', 'Unknown'), entity.get('description', 'No description available.')
    else:
        return "Entity not found", ""

#### OpenAI GPT

OpenAI bietet einen sehr umfangreichen API für seine eigene Modelle. Für kurze Chit-Chat Abfragen ist ein [`Completion`](https://github.com/openai/openai-python/blob/main/src/openai/resources/completions.py) Objekt von [`Responses API`](https://platform.openai.com/docs/api-reference/responses) ausreichend. Um die Ausgabe flexibler definieren zu können, kann man auch `ChatCompletion` mit angepassten Eingaben verwenden. Dieses repräsentiert eine Antwort eines LLM auf konkrete Anfrage als Chatnachricht und wir folgendermaßen erstellt:

In [82]:
import openai

# Define the function to use GPT-3 for generating responses
def generate_gpt_response(question):
    openai.api_key = OPENAI_API_KEY
    print(f'[?] {question}')
    response = openai.chat.completions.create(
        model="gpt-4.1-nano",
        messages=[{ "role": "system", "content": "Answer in two short sentences." },
                  { "role": "user", "content": question }
                  ],
        max_tokens=100
    )
    return response.choices[0].message.content.strip()

Um die Wissenskapazitäten von LLMs erweitern zu können, werden die beiden APIs in ein Pipeline gebunden:

In [83]:
# Main function to integrate the LLM and Knowledge Graph
def knowledge_graph_llm_pipeline(query):
    # Step 1: Query the graph's entities for the specified query
    entity_name, entity_description = query_knowledge_graph(query)
    # Step 2: Generate a question for GPT-3 using the knowledge graph info
    if entity_name != "Entity not found":
        gpt_question = f"What can you tell me about {entity_name}? {entity_description}"
    else:
        gpt_question = f"Tell me more about {query} in general."

    # Step 3: Get a response from the LLM
    gpt_response = generate_gpt_response(gpt_question)
    
    return {
        'Entity': entity_name,
        'Knowledge Graph Info': entity_description,
        'LLM Response': gpt_response
    }

result = knowledge_graph_llm_pipeline(query)
print(" |-< Knowledge Graph Entity:", result['Entity'])
print(" |-< Knowledge Graph Info:", result['Knowledge Graph Info'])
print("[>] LLM Response:", result['LLM Response'])

[?] What can you tell me about NieR: Automata? Video game
 |-< Knowledge Graph Entity: NieR: Automata
 |-< Knowledge Graph Info: Video game
[>] LLM Response: NieR: Automata is an action role-playing game developed by PlatinumGames and published by Square Enix, released in 2017. It features fast-paced combat, a deep storyline exploring themes of consciousness and humanity, and multiple endings.
